In [ ]:
import pandas as pd 
import numpy as np

name = 'KCYM'

all_dirs = {
  f'E-Fergusonii - {name}': f"/home/tlp5359/projects/CC_Sequencing_LLM/BCB/inference-results/{name}/E-fergusonii-to-others/",
  f'E-Coli - {name}': f"/home/tlp5359/projects/CC_Sequencing_LLM/BCB/inference-results/{name}/E-Coli-to-others/",
  f'E-Albertii - {name}': f"/home/tlp5359/projects/CC_Sequencing_LLM/BCB/inference-results/{name}/E-albertii-to-others/",  
}
labels = {f'E-Fergusonii - {name}': "E-fergusonii",
          f'E-Coli - {name}' : "E-Coli",
          f'E-Albertii - {name}': "E-albertii"}
N = [1,2,3]
length_interval=50

acc = {}
cov = {}
for k,v in all_dirs.items():
  all_dfs = []
  # merge folds
  for i in N: 
    tmp_df = pd.read_json(f"{v}/UniRef100-F{i}.json")
    tmp_df = tmp_df[tmp_df['seq_label'] == labels[k]]
    all_dfs.append(tmp_df)
  # take average
  df = pd.concat(all_dfs,ignore_index=True)
  df['bin'] = df['seq_length'].apply(lambda x: (x//length_interval)*length_interval if (x//length_interval)*length_interval <1500 else 1500)
  acc[k] = df[['bin','seq_length','Matching Percentage', 'Matching Percentage full']] #df.groupby('bin')['Matching Percentage'].agg(['mean','std']).reset_index()
  
  # take coverage count
  cov_df = None
  for df in all_dfs:
    df['bin'] = df['seq_length'].apply(lambda x: (x//length_interval)*length_interval if (x//length_interval)*length_interval <1500 else 1500)
    if cov_df is None:
      cov_df = df.groupby('bin')['Matching Percentage'].agg(['count']).reset_index().fillna(0)
      cov_df['cumcount'] = [ 100-v/len(df)*100 for v in cov_df['count'].cumsum()]
    else:
      cov_df_tmp = df.groupby('bin')['Matching Percentage'].agg(['count']).reset_index().fillna(0)
      cov_df_tmp['cumcount'] = [ 100-v/len(df)*100 for v in cov_df_tmp['count'].cumsum()]
      cov_df = pd.concat([cov_df,cov_df_tmp],ignore_index=True)
  cov[k] = cov_df


\textit{E. fergusonii} - KCYM 1


In [ ]:
df.groupby('bin')

In [ ]:
cov_df_tmp

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


k = "E-Fergusonii - KCYM"
fig, _ = plt.subplots(figsize=(8,3))
sns.lineplot(data=acc[k], x='bin', y='Matching Percentage', errorbar=('pi',75), label = "unmask accuracy")
sns.lineplot(data=acc[k], x='bin', y='Matching Percentage full', errorbar=('pi',75), label = "identity")
sns.lineplot(data=cov[k], x='bin', y='cumcount', errorbar=('pi',75), label = "% sequence's length > x", linestyle='--')
plt.axvline(1024, linestyle='--')
plt.xlim([-1,1500])
plt.ylim([0,110])
plt.grid(axis='y')
plt.ylabel('Accuracy [%]')
plt.xlabel("sequence length [residues]")
plt.title(f"{k} - per-sequence accuracy",fontsize=12,pad=25)
plt.legend(bbox_to_anchor=(0.5, 1.08), loc='center', ncol=3)
plt.tight_layout()
plt.show()